Cell 1: Install System and Python Dependencies

In [14]:
# Cell 1: Install System and Python Dependencies
!apt-get update -qq && apt-get install -y libreoffice poppler-utils tesseract-ocr tesseract-ocr-eng -qq

!pip install --quiet --no-cache-dir \
    torch torchvision \
    pdf2image opencv-python-headless \
    pytesseract PyPDF2 \
    python-docx \
    pillow \
    cython pycocotools fvcore \
    sentence-transformers \
    fastapi uvicorn pyngrok \
    psycopg2-binary

!pip install protobuf==3.20.3 google-api-core==1.34.1
!pip install faiss-cpu --upgrade
!pip install --quiet -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'
!pip install --quiet "layoutparser[ocr]"
!pip install tensorflow==2.4.1  # Add this line
!pip install spacy

# Verify installations
import torch
import layoutparser
import detectron2
import faiss
import tensorflow as tf
print(f"Torch version: {torch.__version__}")
print(f"Layoutparser version: {layoutparser.__version__}")
print(f"Detectron2 version: {detectron2.__version__}")
print(f"FAISS version: {faiss.__version__}")
print(f"TensorFlow version: {tf.__version__}")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.4.1
Torch version: 2.6.0+cu124
Layoutparser version: 0.3.4
Detectron2 version: 0.6
FAISS version: 1.10.0
TensorFlow version: 2.18.0


Imports and Initial Setup

In [28]:
# Cell 2: Imports and Initial Setup
# -*- coding: utf-8 -*-
import os
import subprocess
from pathlib import Path
from datetime import datetime
import torch
import layoutparser as lp
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.model_zoo import model_zoo
from pdf2image import convert_from_path
import cv2
import numpy as np
import pytesseract
from PIL import Image
from sentence_transformers import SentenceTransformer
import faiss
import psycopg2
from psycopg2.extras import execute_values
from psycopg2.extras import execute_batch
from google.colab import drive
import hashlib
import re
import traceback
import spacy

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Mount Google Drive
drive.mount('/content/drive')

# Directory setup
BASE_DIR = Path("/content/drive/My Drive/lifesciences")
SOURCE_DIR = BASE_DIR / "training_documents"
PDF_STAGING_DIR = BASE_DIR / "staging/pdf"
IMG_STAGING_DIR = BASE_DIR / "post_label/images"
ANNOTATED_DIR = BASE_DIR / "staging/annotated"
MODELS_DIR = BASE_DIR / "models"
RETRAINED_STAGING_DIR = BASE_DIR / "retrained_staging"
RETRAINED_ANNOTATED_DIR = RETRAINED_STAGING_DIR / "annotated"
FAISS_BASE_DIR = BASE_DIR / "faiss_indexes"
COCO_JSON_PATH = BASE_DIR / "post_label/result.json"
RETRAINED_MODELS_DIR = BASE_DIR / "retrained_staging/models"

for directory in [SOURCE_DIR, PDF_STAGING_DIR, IMG_STAGING_DIR, ANNOTATED_DIR, MODELS_DIR, RETRAINED_STAGING_DIR, RETRAINED_ANNOTATED_DIR, FAISS_BASE_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

Image.MAX_IMAGE_PIXELS = 100000000  # Increase max image pixel limit

# Database configuration
DB_CONFIG = {
    "dbname": "neondb",
    "user": "neondb_owner",
    "password": "npg_DU3Vxoi6cCIu",
    "host": "ep-shy-recipe-a406b4wg-pooler.us-east-1.aws.neon.tech",
    "port": "5432",
    "sslmode": "require"
}

# Global variables (initialized lazily)
sentence_model = None
model = None
ocr_agent = None
COLORS = {"title": (255, 0, 0), "text": (0, 255, 0), "list": (0, 0, 255), "table": (255, 255, 0), "figure": (255, 0, 255)}
LABEL_MAP = {0: "figure", 1: "list", 2: "table", 3: "text", 4: "title"}

Using device: cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cell 3: Utility Functions

In [29]:
# Cell 3: Utility Functions
def setup_db_connection():
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO batch_runs (started_at, processed_files, batch_status)
        VALUES (%s, 0, 'RUNNING')
        RETURNING id
    """, (datetime.now().isoformat(),))
    batch_id = cursor.fetchone()[0]
    conn.commit()
    print(f"Started batch run: batch_run_id={batch_id}")
    return conn, cursor, batch_id

def update_run_status(conn, cursor, batch_id, status, processed_files=0):
    try:
        cursor.execute("""
            UPDATE batch_runs
            SET completed_at = %s, processed_files = %s, batch_status = %s
            WHERE id = %s
        """, (datetime.now().isoformat(), processed_files, status, batch_id))
        conn.commit()
        print(f"Updated batch run {batch_id} to {status} with {processed_files} files processed")
    except psycopg2.Error as e:
        print(f"Database error during status update: {e}")
        conn.rollback()
        cursor.execute("""
            UPDATE batch_runs
            SET completed_at = %s, processed_files = %s, batch_status = %s
            WHERE id = %s
        """, (datetime.now().isoformat(), processed_files, status, batch_id))
        conn.commit()
        print(f"Retried and updated batch run {batch_id} to {status} after rollback")

def preprocess_name_for_hash(name):
    name = re.sub(r'^\d+\s*', '', name).strip()
    return re.sub(r'[^a-z0-9]', '', name.lower())

def preprocess_name_for_insert(name):
    return re.sub(r'^\d+\s*', '', name).strip()

def get_faiss_index_path(urs_name):
    hashed_name = hashlib.md5(urs_name.encode()).hexdigest()
    return FAISS_BASE_DIR / f"{hashed_name}.index"

Cell 4: Model Training Function

In [30]:
# Cell 4: Model Training Function
def train_layoutparser_model(json_path, image_dir):
    dataset_name = "custom_layout_dataset"
    if dataset_name in DatasetCatalog:
        DatasetCatalog.remove(dataset_name)
        MetadataCatalog.remove(dataset_name)
    register_coco_instances(dataset_name, {}, str(json_path), image_dir)
    MetadataCatalog.get(dataset_name).thing_classes = list(LABEL_MAP.values())

    cfg = get_cfg()
    pretrained_model_path = MODELS_DIR / "model_final.pth"
    if pretrained_model_path.exists():
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = str(pretrained_model_path)
    else:
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")

    cfg.MODEL.DEVICE = str(device)
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 300
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(LABEL_MAP)
    cfg.OUTPUT_DIR = str(RETRAINED_MODELS_DIR)

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    model_path = Path(cfg.OUTPUT_DIR) / "model_final.pth"
    config_path = Path(cfg.OUTPUT_DIR) / "config.yaml"
    with open(config_path, "w") as f:
        f.write(cfg.dump())

    global model
    model = lp.Detectron2LayoutModel(
        config_path=str(config_path),
        model_path=str(model_path),
        label_map=LABEL_MAP,
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
        device=str(device)
    )
    print(f"Model trained and saved to {model_path}")
    return model

Cell 5: Layout Processing Functions

In [31]:
# Cell 5: Layout Processing Functions

import spacy
spacy.cli.download("en_core_web_lg")  # Downloads the model


nlp = spacy.load("en_core_web_lg")  # You may need to run: python -m spacy download en_core_web_sm

def preprocess_text(text):
    # Insert a period and space before numbers that likely start new sentences
    text = re.sub(r'(\d+)\s+([A-Z])', r'. \1 \2', text)
    # Replace multiple spaces with a single space
    text = ' '.join(text.split())
    return text

def clean_and_split_sentences(text):
    # Preprocess the text to help spaCy detect sentence boundaries
    preprocessed_text = preprocess_text(text)

    # Process the text with spaCy
    doc = nlp(preprocessed_text)

    # Initialize list for cleaned sentences
    cleaned_sentences = []

    # Define special characters to remove from the start
    special_chars = r'[|!@#$%^&*()_+=[\]{}:;"\'<>,/~`\\-]'

    # Iterate through detected sentences
    for sent in doc.sents:
        # Get the sentence text and clean it
        sentence = sent.text.strip()

        # Replace all '-' with a single space
        sentence = sentence.replace('-', ' ')

        # Remove leading numbers and special characters
        sentence = re.sub(f'^{special_chars}+', '', sentence.strip())
        sentence = re.sub(r'^\d+', '', sentence.strip())

        # Remove extra spaces
        sentence = ' '.join(sentence.split())

        # Skip if sentence is empty
        if not sentence:
            continue

        sentence = sentence.strip()
        # Append a full stop if not already present
        if sentence[-1] not in '.!?':
            sentence = sentence.strip()+'.'

        # Filter for meaningful sentences
        tokens = nlp(sentence)
        has_verb = any(token.pos_ == "VERB" for token in tokens)
        is_long_enough = len(sentence) > 15

        if has_verb and is_long_enough:
            cleaned_sentences.append(sentence)

    return cleaned_sentences


def lazy_load_models():
    global model, ocr_agent
    if ocr_agent is None:
        ocr_agent = lp.TesseractAgent(languages='eng', config='--psm 6 --oem 1')
        print("OCR agent loaded")
    if model is None:
        model_path = RETRAINED_MODELS_DIR / "model_final.pth"
        config_path = RETRAINED_MODELS_DIR / "config.yaml"
        if model_path.exists() and config_path.exists():
            model = lp.Detectron2LayoutModel(
                config_path=str(config_path),
                model_path=str(model_path),
                label_map=LABEL_MAP,
                extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                device=str(device)
            )
            print(f"Loaded model from {model_path}")
        else:
            raise ValueError("Model not found. Train it first.")

def is_overlapping(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect2[2] <= rect1[0] or
                rect1[3] <= rect2[1] or rect2[3] <= rect1[1])

def process_layout(image_path, urs_name, conn, cursor, batch_id):
    global model
    lazy_load_models()
    if model is None:
        raise ValueError("Model not loaded.")

    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")

    layout = model.detect(image)
    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])
    skipped_blocks = []
    list_table_regions = [(b.type.lower(), list(map(int, b.coordinates))) for b in sorted_layout if b.type.lower() in ["list", "table"]]
    current_section = "notitle"
    block_number = 1

    # Precompute list and table contents
    list_table_contents = {}
    for block_type, coords in list_table_regions:
        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        content = ocr_agent.detect(cropped) or ""
        if block_type == "list":
            content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
        elif block_type == "table":
            content = '\n'.join('-'.join(col.strip() for col in row.split() if col.strip()) for row in content.split('\n') if row.strip()) or "Table Content"
        list_table_contents[tuple(coords)] = content.strip()
        del cropped

    # Precompute figure contents
    figure_regions = [(b.type.lower(), list(map(int, b.coordinates))) for b in sorted_layout if b.type.lower() == "figure"]
    for block_type, coords in figure_regions:
        if tuple(coords) not in list_table_contents:  # Avoid overwriting if already processed
            cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
            content = ocr_agent.detect(cropped) or "Figure Content"
            list_table_contents[tuple(coords)] = content.strip()
            del cropped

    # Calculate urs_hash
    urs_hash = hashlib.md5(preprocess_name_for_hash(urs_name).encode()).hexdigest()

    # Insert or get URS ID with urs_hash
    cursor.execute("INSERT INTO urs (name, urs_hash) VALUES (%s, %s) ON CONFLICT (name) DO NOTHING RETURNING id", (urs_name, urs_hash))
    row = cursor.fetchone()
    urs_id = row[0] if row else (cursor.execute("SELECT id FROM urs WHERE name = %s", (urs_name,)) or cursor.fetchone()[0])
    conn.commit()

    # Initialize default section "notitle"
    section_hash = hashlib.md5(preprocess_name_for_hash(current_section).encode()).hexdigest()
    cursor.execute("SELECT id FROM sections WHERE section_hash = %s", (section_hash,))
    row = cursor.fetchone()
    if not row:
        cursor.execute("INSERT INTO sections (name, section_hash) VALUES (%s, %s) RETURNING id", (current_section, section_hash))
        section_id = cursor.fetchone()[0]
    else:
        section_id = row[0]
    cursor.execute("INSERT INTO urs_section_mapping (urs_id, section_id) VALUES (%s, %s) ON CONFLICT DO NOTHING RETURNING urs_section_id", (urs_id, section_id))
    row = cursor.fetchone()
    urs_section_id = row[0] if row else (cursor.execute("SELECT urs_section_id FROM urs_section_mapping WHERE urs_id = %s AND section_id = %s", (urs_id, section_id)) or cursor.fetchone()[0])
    conn.commit()

    # Process blocks and write to DB immediately
    for block in sorted_layout:
        coords = list(map(int, block.coordinates))
        block_type = block.type.lower()
        overlapping_region = next((r for r in list_table_regions if is_overlapping(coords, r[1])), None)
        if block_type in ["title", "text"] and overlapping_region:
            parent_type, parent_coords = overlapping_region
            cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
            skipped_content = ocr_agent.detect(cropped) or ""
            if block_type == "title":
                skipped_content = skipped_content or "Untitled"
            skipped_blocks.append((
                batch_id, parent_type, block_type, current_section,
                list_table_contents[tuple(parent_coords)][:997] + "..." if len(list_table_contents[tuple(parent_coords)]) > 1000 else list_table_contents[tuple(parent_coords)],
                skipped_content.strip()[:997] + "..." if len(skipped_content.strip()) > 1000 else skipped_content.strip(),
                parent_coords, coords
            ))
            del cropped
            continue

        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        coords_tuple = tuple(coords)
        if block_type in ["list", "table", "figure"]:
            content = list_table_contents.get(coords_tuple, "Unknown Content")  # Safe access with fallback
            content_lines = clean_and_split_sentences(content)
        elif block_type == "title":
            content = ocr_agent.detect(cropped) or "Untitled"
            current_section = preprocess_name_for_insert(content)
            section_hash = hashlib.md5(preprocess_name_for_hash(content).encode()).hexdigest()
            cursor.execute("SELECT id FROM sections WHERE section_hash = %s", (section_hash,))
            row = cursor.fetchone()
            if not row:
                cursor.execute("INSERT INTO sections (name, section_hash) VALUES (%s, %s) RETURNING id", (current_section, section_hash))
                section_id = cursor.fetchone()[0]
            else:
                section_id = row[0]
            cursor.execute("INSERT INTO urs_section_mapping (urs_id, section_id) VALUES (%s, %s) ON CONFLICT DO NOTHING RETURNING urs_section_id", (urs_id, section_id))
            row = cursor.fetchone()
            urs_section_id = row[0] if row else (cursor.execute("SELECT urs_section_id FROM urs_section_mapping WHERE urs_id = %s AND section_id = %s", (urs_id, section_id)) or cursor.fetchone()[0])
            del cropped
            continue
        else:
            content = ocr_agent.detect(cropped) or ""
            content_lines = [content]
        del cropped

        # Write content blocks immediately
        content_values = []
        for line in content_lines:
            if line.strip():
                content_values.append((
                    batch_id, urs_section_id, block_number, block_type, line.strip(),
                    coords[0], coords[1], coords[2], coords[3], datetime.now().isoformat(), None
                ))
                block_number += 1
        if content_values:
            execute_values(cursor, """
                INSERT INTO content_blocks (batch_run_id, urs_section_id, block_number, content_type, content,
                                           coord_x1, coord_y1, coord_x2, coord_y2, created_at, faiss_index_id)
                VALUES %s
                ON CONFLICT (batch_run_id, urs_section_id, block_number) DO UPDATE SET
                    content_type = EXCLUDED.content_type, content = EXCLUDED.content,
                    coord_x1 = EXCLUDED.coord_x1, coord_y1 = EXCLUDED.coord_y1,
                    coord_x2 = EXCLUDED.coord_x2, coord_y2 = EXCLUDED.coord_y2,
                    created_at = EXCLUDED.created_at, faiss_index_id = EXCLUDED.faiss_index_id
            """, content_values)
            conn.commit()

        x1, y1, x2, y2 = coords
        color = COLORS.get(block_type, (255, 255, 255))
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{block_type} #{block_number-1}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    annotated_path = RETRAINED_ANNOTATED_DIR / f"{Path(image_path).stem}_reannotated.png"
    cv2.imwrite(annotated_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    del image

    # Write skipped blocks
    if skipped_blocks:
        execute_values(cursor, """
            INSERT INTO skipped_block_items (batch_run_id, parent_block_type, skipped_block_type, section_name,
                                             parent_block_content, skipped_block_content, parent_block_coordinates, skipped_block_coordinates)
            VALUES %s
        """, skipped_blocks)
        conn.commit()

    return urs_section_id, block_number - 1

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Cell 6: Embedding and Storage Functions

In [32]:
# Cell 6: Embedding and Storage Functions
def initialize_embedding_tools(urs_name):
    global sentence_model
    if sentence_model is None:
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
        print("SentenceTransformer loaded")

    faiss_index_path = get_faiss_index_path(urs_name)
    embedding_dim = sentence_model.encode("test").shape[0]
    if faiss_index_path.exists():
        faiss_index = faiss.read_index(str(faiss_index_path))
        print(f"Loaded FAISS index for {urs_name} from {faiss_index_path}")
    else:
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        print(f"Initialized new FAISS index for {urs_name}")
    return faiss_index, faiss_index_path

def append_to_faiss(urs_name, urs_section_id, block_count, conn, cursor, batch_id):
    faiss_index, faiss_index_path = initialize_embedding_tools(urs_name)

    # Fetch recently added content blocks
    cursor.execute("""
        SELECT cb.id, cb.content, s.name
        FROM content_blocks cb
        JOIN urs_section_mapping usm ON cb.urs_section_id = usm.urs_section_id
        JOIN sections s ON usm.section_id = s.id
        WHERE cb.batch_run_id = %s AND cb.urs_section_id = %s AND cb.faiss_index_id IS NULL
    """, (batch_id, urs_section_id))
    rows = cursor.fetchall()

    if rows:
        query_strings = [f"{row[2]}||{row[1]}" for row in rows]
        embeddings = sentence_model.encode(query_strings, convert_to_numpy=True)
        faiss_index.add(embeddings)
        index_positions = list(range(faiss_index.ntotal - len(rows), faiss_index.ntotal))

        # Update faiss_index_id in content_blocks
        update_values = [(row[0], idx) for idx, row in zip(index_positions, rows)]
        execute_batch(cursor, """
            UPDATE content_blocks SET faiss_index_id = %s WHERE id = %s
        """, update_values)
        conn.commit()
        faiss.write_index(faiss_index, str(faiss_index_path))
        print(f"Updated FAISS index for {urs_name} with {len(rows)} entries")

        del embeddings, query_strings
    del rows

Cell 7: Main Processing Logic

In [33]:
# Cell 7: Main Processing Logic
def convert_docx_to_pdf(docx_path, pdf_output_dir):
    pdf_path = pdf_output_dir / f"{Path(docx_path).stem}.pdf"
    try:
        subprocess.run(["libreoffice", "--headless", "--convert-to", "pdf", str(docx_path), "--outdir", str(pdf_output_dir)], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return pdf_path
    except subprocess.CalledProcessError as e:
        print(f"Error converting {docx_path} to PDF: {e}")
        return None

def extract_images_from_pdf(pdf_path, image_output_dir):
    try:
        images = convert_from_path(pdf_path, output_folder=image_output_dir, fmt="png")
        image_paths = [Path(image.filename) for image in images]
        return image_paths
    except Exception as e:
        print(f"Error extracting images from {pdf_path}: {e}")
        return []

def process_images():
    conn, cursor, batch_id = setup_db_connection()
    processed_files = 0

    try:
        source_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(('.docx', '.pdf'))]
        for source_file in source_files:
            source_file_path = SOURCE_DIR / source_file
            urs_name = Path(source_file).stem
            pdf_path = source_file_path if source_file.endswith('.pdf') else convert_docx_to_pdf(source_file_path, PDF_STAGING_DIR)

            if pdf_path and pdf_path.exists():
                image_paths = extract_images_from_pdf(pdf_path, IMG_STAGING_DIR)
                for image_path in image_paths:
                    print(f"Processing {image_path} for URS {urs_name}")
                    urs_section_id, block_count = process_layout(str(image_path), urs_name, conn, cursor, batch_id)
                    append_to_faiss(urs_name, urs_section_id, block_count, conn, cursor, batch_id)
                    os.remove(image_path)
                if source_file.endswith('.docx') and pdf_path.exists():
                    os.remove(pdf_path)
                processed_files += 1
            else:
                print(f"Skipping {source_file} due to conversion failure")

        update_run_status(conn, cursor, batch_id, 'SUCCESS', processed_files)
    except Exception as e:
        print(f"Error: {e}")
        print(traceback.format_exc())
        update_run_status(conn, cursor, batch_id, 'FAILED', processed_files)
        raise
    finally:
        cursor.close()
        conn.close()

# Main Execution
process_images()

Started batch run: batch_run_id=6
Processing /content/drive/My Drive/lifesciences/post_label/images/2bc4a9cc-0511-4e28-9439-4daaf78cb1fb-1.png for URS Pharma_URS_Enhanced
OCR agent loaded
Loaded model from /content/drive/My Drive/lifesciences/retrained_staging/models/model_final.pth
SentenceTransformer loaded
Initialized new FAISS index for Pharma_URS_Enhanced
Processing /content/drive/My Drive/lifesciences/post_label/images/2bc4a9cc-0511-4e28-9439-4daaf78cb1fb-2.png for URS Pharma_URS_Enhanced
Initialized new FAISS index for Pharma_URS_Enhanced
Processing /content/drive/My Drive/lifesciences/post_label/images/47fd018b-c6ce-490c-a8c9-6b15162f3c8f-01.png for URS ProtonGlow_Test_URS
Loaded FAISS index for ProtonGlow_Test_URS from /content/drive/My Drive/lifesciences/faiss_indexes/28481597252c1a8c5171213add762432.index
Processing /content/drive/My Drive/lifesciences/post_label/images/47fd018b-c6ce-490c-a8c9-6b15162f3c8f-02.png for URS ProtonGlow_Test_URS
Loaded FAISS index for ProtonGlow_

Cell 8: Database Schema (Reference)

In [34]:
# Cell 8: Database Schema (Reference)
"""
-- Drop existing tables and sequences
DROP TABLE IF EXISTS skipped_block_items CASCADE;
DROP TABLE IF EXISTS content_blocks CASCADE;
DROP TABLE IF EXISTS urs_section_mapping CASCADE;
DROP TABLE IF EXISTS sections CASCADE;
DROP TABLE IF EXISTS batch_runs CASCADE;
DROP TABLE IF EXISTS urs CASCADE;

DROP SEQUENCE IF EXISTS skipped_block_items_id_seq;
DROP SEQUENCE IF EXISTS content_blocks_id_seq;
DROP SEQUENCE IF EXISTS urs_section_mapping_urs_section_id_seq;
DROP SEQUENCE IF EXISTS sections_id_seq;
DROP SEQUENCE IF EXISTS batch_runs_id_seq;
DROP SEQUENCE IF EXISTS urs_id_seq;

-- Create tables
CREATE TABLE urs (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL UNIQUE,
    urs_hash VARCHAR(32) NOT NULL
);

CREATE TABLE batch_runs (
    id SERIAL PRIMARY KEY,
    started_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    completed_at TIMESTAMP,
    processed_files INTEGER DEFAULT 0,
    batch_status VARCHAR(20) DEFAULT 'RUNNING' CHECK (batch_status IN ('RUNNING', 'SUCCESS', 'FAILED'))
);

CREATE TABLE sections (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    section_hash VARCHAR(32) NOT NULL UNIQUE
);

CREATE TABLE urs_section_mapping (
    urs_section_id SERIAL PRIMARY KEY,
    urs_id INTEGER NOT NULL,
    section_id INTEGER NOT NULL,
    UNIQUE (urs_id, section_id),
    FOREIGN KEY (urs_id) REFERENCES urs(id),
    FOREIGN KEY (section_id) REFERENCES sections(id)
);

CREATE TABLE content_blocks (
    id SERIAL PRIMARY KEY,
    batch_run_id INTEGER NOT NULL,
    urs_section_id INTEGER NOT NULL,
    block_number INTEGER NOT NULL,
    content_type VARCHAR(50) NOT NULL,
    content TEXT,
    coord_x1 INTEGER,
    coord_y1 INTEGER,
    coord_x2 INTEGER,
    coord_y2 INTEGER,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    faiss_index_id INTEGER,
    UNIQUE (batch_run_id, urs_section_id, block_number),
    FOREIGN KEY (batch_run_id) REFERENCES batch_runs(id),
    FOREIGN KEY (urs_section_id) REFERENCES urs_section_mapping(urs_section_id)
);

CREATE TABLE skipped_block_items (
    id SERIAL PRIMARY KEY,
    batch_run_id INTEGER NOT NULL,
    parent_block_type VARCHAR(20) NOT NULL,
    skipped_block_type VARCHAR(20) NOT NULL,
    section_name VARCHAR(255),
    parent_block_content VARCHAR(1000),
    skipped_block_content VARCHAR(1000),
    parent_block_coordinates INTEGER[],
    skipped_block_coordinates INTEGER[],
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (batch_run_id) REFERENCES batch_runs(id)
);
"""

"\n-- Drop existing tables and sequences\nDROP TABLE IF EXISTS skipped_block_items CASCADE;\nDROP TABLE IF EXISTS content_blocks CASCADE;\nDROP TABLE IF EXISTS urs_section_mapping CASCADE;\nDROP TABLE IF EXISTS sections CASCADE;\nDROP TABLE IF EXISTS batch_runs CASCADE;\nDROP TABLE IF EXISTS urs CASCADE;\n\nDROP SEQUENCE IF EXISTS skipped_block_items_id_seq;\nDROP SEQUENCE IF EXISTS content_blocks_id_seq;\nDROP SEQUENCE IF EXISTS urs_section_mapping_urs_section_id_seq;\nDROP SEQUENCE IF EXISTS sections_id_seq;\nDROP SEQUENCE IF EXISTS batch_runs_id_seq;\nDROP SEQUENCE IF EXISTS urs_id_seq;\n\n-- Create tables\nCREATE TABLE urs (\n    id SERIAL PRIMARY KEY,\n    name VARCHAR(255) NOT NULL UNIQUE,\n    urs_hash VARCHAR(32) NOT NULL\n);\n\nCREATE TABLE batch_runs (\n    id SERIAL PRIMARY KEY,\n    started_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n    completed_at TIMESTAMP,\n    processed_files INTEGER DEFAULT 0,\n    batch_status VARCHAR(20) DEFAULT 'RUNNING' CHECK (batch_status IN ('RUNN